# pistachio


In [1]:
# convert arff to parquet

from scipy.io import arff 
import pandas as pd
import os

arff_filepath = './data/Pistachio_Dataset/Pistachio_16_Features_Dataset/Pistachio_16_Features_Dataset.arff'
parquet_path = './data/pistachio_16.snappy.pqt'

def arff_to_parquet(input_arff: str, output_parquet: str):
    """convert arff file to parquet"""
    if not os.path.exists(input_arff):
        raise ValueError(f"input file '{input_arff}' does not exist")
    data, meta = arff.loadarff(input_arff)
    print("arff metadata")
    print(meta)
    df = pd.DataFrame(data)
    df['Class'] = df['Class'].astype(str)
    df.to_parquet(output_parquet)
##################

if not os.path.exists(parquet_path):
    print("converting arff to parquet")
    arff_to_parquet(arff_filepath, parquet_path)


In [2]:
# Dataset: Pistachio_16_Features_Dataset
# 	AREA's type is numeric
# 	PERIMETER's type is numeric
# 	MAJOR_AXIS's type is numeric
# 	MINOR_AXIS's type is numeric
# 	ECCENTRICITY's type is numeric
# 	EQDIASQ's type is numeric
# 	SOLIDITY's type is numeric
# 	CONVEX_AREA's type is numeric
# 	EXTENT's type is numeric
# 	ASPECT_RATIO's type is numeric
# 	ROUNDNESS's type is numeric
# 	COMPACTNESS's type is numeric
# 	SHAPEFACTOR_1's type is numeric
# 	SHAPEFACTOR_2's type is numeric
# 	SHAPEFACTOR_3's type is numeric
# 	SHAPEFACTOR_4's type is numeric
# 	Class's type is nominal, range is ('Kirmizi_Pistachio', 'Siit_Pistachio')

## Load Data
load data from parquet, stratify split to train and test

In [2]:
from typing import List
import numpy as np
from sklearn.model_selection import train_test_split

def split_data(
        input_parquet: str, 
        train_filename: str,
        test_filename: str,
        label_column: str,
        test_fraction: float=0.2,
        seed: int=42):
    """stratify sample the data"""
    # set seed
    # np.random.seed(seed)
    in_df = pd.read_parquet(input_parquet)
    y = in_df.pop(label_column)
    x_train, x_test, y_train, y_test = train_test_split(
        in_df, 
        y, 
        random_state=seed, 
        stratify=y, 
        test_size=test_fraction)
    # reattach labels
    x_train[label_column] = y_train
    x_test[label_column] = y_test
    print(f'x_train shape = {x_train.shape}')
    print(f'y_train shape = {y_train.shape}')
    print(f'x_test shape = {x_test.shape}')
    print(f'y_test shape = {y_test.shape}')
    # write data
    x_train.to_parquet(train_filename)
    x_test.to_parquet(test_filename)
##############################

train_path = './data/pistachio_train.pqt'
test_path = './data/pistachio_test.pqt'
split_seed = 41
label_column = 'Class'
test_fraction = 0.2
input_data_schema_path = "./data/pistachio_schema.json"

if not (os.path.exists(train_path) and os.path.exists(test_path)):
    split_data(
        parquet_path,
        train_path,
        test_path,
        label_column=label_column,
        test_fraction=test_fraction,
        seed=split_seed)
    print('split train and test data')


## Validate Data

In [3]:
#! pip install pandera[io]

In [4]:
# very quick EDA/summary on the train data
train = pd.read_parquet(train_path)
summary = train.describe(include='all')
summary

,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,ROUNDNESS,COMPACTNESS,SHAPEFACTOR_1,SHAPEFACTOR_2,SHAPEFACTOR_3,SHAPEFACTOR_4,Class
count,1718.000000,1718.000000,1718.000000,1718.000000,1718.000000,1718.000000,1718.000000,1718.000000,1718.000000,1718.000000,1718.000000,1718.000000,1718.000000,1718.000000,1718.000000,1718.000000,1718
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kirmizi_Pistachio
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,985
mean,79990.176368,1423.800903,446.184575,238.404956,0.840033,318.003064,0.940279,85048.962165,0.715327,1.897053,0.571019,0.713397,0.005698,0.003016,0.510931,0.955492,NaN
std,13075.738130,375.345641,32.531725,30.372990,0.048895,26.853806,0.050205,13180.036678,0.053204,0.239194,0.212615,0.044681,0.000825,0.000333,0.064074,0.050951,NaN
min,29808.000000,858.363000,320.344500,133.509600,0.504900,194.814600,0.588000,37935.000000,0.427200,1.158500,0.062800,0.476000,0.004000,0.002400,0.226600,0.620400,NaN
25%,71859.500000,1169.588750,426.337050,217.925450,0.817225,302.480300,0.920025,76334.000000,0.685225,1.735225,0.375375,0.682225,0.005200,0.002800,0.465425,0.944000,NaN
50%,79997.500000,1261.599000,448.897750,236.299450,0.849200,319.148850,0.954500,85097.000000,0.725800,1.893500,0.644100,0.710850,0.005600,0.003000,0.505350,0.973300,NaN
75%,89018.000000,1603.212000,468.631300,257.292375,0.875100,336.661875,0.976775,93919.000000,0.753575,2.066650,0.744500,0.742300,0.006100,0.003200,0.551000,0.987200,NaN


In [5]:

import pandera as pa
# schema = pa.infer_schema(train)
# print(schema)
from pandera import Check, Column, DataFrameSchema
# define schema based on inspecting the training data above
if not os.path.exists(input_data_schema_path):
# if True:
    print("creating schema")
    schema = DataFrameSchema(
        columns={
            "AREA": Column(
                dtype="float64",
                checks=[
                    Check.greater_than_or_equal_to(min_value=100.0),
                    Check.less_than_or_equal_to(max_value=1e6)
                ],
                nullable=False
            ),
            "PERIMETER": Column(
                dtype="float64",
                checks=[
                    Check.greater_than_or_equal_to(min_value=100.0),
                    Check.less_than_or_equal_to(max_value=1e6)
                ],
                nullable=False
            ),
            "MAJOR_AXIS": Column(
                dtype="float64",
                checks=[
                    Check.greater_than_or_equal_to(min_value=10.0),
                    Check.less_than_or_equal_to(max_value=1e6)
                ],
                nullable=False
            ),
            "MINOR_AXIS": Column(
                dtype="float64",
                checks=[
                    Check.greater_than_or_equal_to(min_value=10.0),
                    Check.less_than_or_equal_to(max_value=1e6)
                ],
                nullable=False
            ),
            "ECCENTRICITY": Column(
                dtype="float64",
                checks=[
                    Check.greater_than_or_equal_to(min_value=0.0),
                    Check.less_than_or_equal_to(max_value=1.0)
                ],
                nullable=False
            ),
            "EQDIASQ": Column(
                dtype="float64",
                checks=[
                    Check.greater_than_or_equal_to(min_value=100.0),
                    Check.less_than_or_equal_to(max_value=1e6)
                ],
                nullable=False
            ),
            "SOLIDITY": Column(
                dtype="float64",
                checks=[
                    Check.greater_than_or_equal_to(min_value=0.0),
                    Check.less_than_or_equal_to(max_value=1.0)
                ],
                nullable=False
            ),
            "CONVEX_AREA": Column(
                dtype="float64",
                checks=[
                    Check.greater_than_or_equal_to(min_value=1000.0),
                    Check.less_than_or_equal_to(max_value=1e6)
                ],
                nullable=False
            ),
            "EXTENT": Column(
                dtype="float64",
                checks=[
                    Check.greater_than_or_equal_to(min_value=0.0),
                    Check.less_than_or_equal_to(max_value=1.0)
                ],
                nullable=False
            ),
            "ASPECT_RATIO": Column(
                dtype="float64",
                checks=[
                    Check.greater_than_or_equal_to(min_value=0.0),
                    Check.less_than_or_equal_to(max_value=100.0)
                ],
                nullable=False
            ),
            "ROUNDNESS": Column(
                dtype="float64",
                checks=[
                    Check.greater_than_or_equal_to(min_value=0.0),
                    Check.less_than_or_equal_to(max_value=1.0)
                ],
                nullable=False
            ),
            "COMPACTNESS": Column(
                dtype="float64",
                checks=[
                    Check.greater_than_or_equal_to(min_value=0.0),
                    Check.less_than_or_equal_to(max_value=1.0)
                ],
                nullable=False
            ),
            "SHAPEFACTOR_1": Column(
                dtype="float64",
                checks=[
                    Check.greater_than_or_equal_to(min_value=0.0),
                    Check.less_than_or_equal_to(max_value=1.0)
                ],
                nullable=False
            ),
            "SHAPEFACTOR_2": Column(
                dtype="float64",
                checks=[
                    Check.greater_than_or_equal_to(min_value=0.0),
                    Check.less_than_or_equal_to(max_value=1.0)
                ],
                nullable=False
            ),
            "SHAPEFACTOR_3": Column(
                dtype="float64",
                checks=[
                    Check.greater_than_or_equal_to(min_value=0.0),
                    Check.less_than_or_equal_to(max_value=1.0)
                ],
                nullable=False
            ),
            "SHAPEFACTOR_4": Column(
                dtype="float64",
                checks=[
                    Check.greater_than_or_equal_to(min_value=0.0),
                    Check.less_than_or_equal_to(max_value=1.0)
                ],
                nullable=False
            ),
            "Class": Column(
                dtype="object",
                checks=[
                    Check.isin(['Siit_Pistachio', 'Kirmizi_Pistachio'])
                ],
                nullable=False
            )
        }
    )
    print(schema)
    schema.to_json(input_data_schema_path)
    print(f"wrote schema to {input_data_schema_path}")


In [6]:

# Validate Data (dtypes, count nulls)
# preprocess - feature engineering, cast class to 1/0
# tune - hyperopt or bayes_opt
# stash metadata/experiment results
# train with best parameters
# evaluate
# run inference on dummy "new" data
from pandera import DataFrameSchema
def validate_data(in_df: pd.DataFrame, schema_file: str) -> pd.DataFrame:
    """check input data, count nulls, basic stats"""
    # load schema
    the_schema = DataFrameSchema.from_json(schema_file)
    the_schema.validate(in_df)

    
    
    # summary = in_df.describe(include='all')
    # # check for entirely missing columns
    # entirely_missing = [x for x in in_df.columns if summary.loc['count', x] == 0]
    # if entirely_missing:
    #     raise ValueError(f'following columns in supplied data are missing: {entirely_missing}')
    # # check that columns have more than one unique value
    # single_value_columns = [x for x in in_df.columns if summary.loc['unique', column] == 1]
    # if entirely_missing:
    #     raise ValueError(f'following columns in supplied data are missing: {entirely_missing}')

validate_data(train, input_data_schema_path)

print("no errors, data looks good")



# train['Class'] = train.Class.astype('category')
# train.Class.cat.codes

# https://pandas.pydata.org/docs/user_guide/categorical.html#controlling-behavior


no errors, data looks good


In [9]:
from pandas.api.types import CategoricalDtype
def preprocess(in_raw_df: pd.DataFrame) -> pd.DataFrame:
    """preprocess the data, do any cleaning, feature engineering, etc"""
    out_df = in_raw_df.copy()

    #cross some features
    out_df['SOLIDITY_MAJOR'] = out_df.SOLIDITY*out_df.MAJOR_AXIS

    # reorder
    cols = [x for x in out_df.columns if x != 'Class']
    out_df = out_df[cols + ['Class']]

    # convert Class to categorical
    class_type = CategoricalDtype(categories=['Siit_Pistachio', 'Kirmizi_Pistachio'])
    out_df.Class = out_df.Class.astype(class_type)
    # create a binary column
    out_df['Target'] = out_df.Class.cat.codes

    return out_df

train_proc = preprocess(train)
train_proc.head()


    
    

,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,ROUNDNESS,COMPACTNESS,SHAPEFACTOR_1,SHAPEFACTOR_2,SHAPEFACTOR_3,SHAPEFACTOR_4,SOLIDITY_MAJOR,Class,Target
1903,101005.0,1382.8000,476.3697,279.0725,0.8104,358.6134,0.9478,106565.0,0.7732,1.7070,0.6638,0.7528,0.0047,0.0028,0.5667,0.9674,451.503202,Siit_Pistachio,0
1168,96240.0,1427.5699,476.6801,264.1641,0.8324,350.0522,0.9703,99186.0,0.7946,1.8045,0.5934,0.7344,0.0050,0.0027,0.5393,0.9731,462.522701,Kirmizi_Pistachio,1
716,48509.0,1020.7170,439.9729,143.3477,0.9454,248.5228,0.9792,49537.0,0.7156,3.0693,0.5851,0.5649,0.0091,0.0030,0.3191,0.9793,430.821464,Kirmizi_Pistachio,1
1570,100634.0,1260.0811,481.9576,266.3690,0.8334,357.9542,0.9925,101396.0,0.7059,1.8094,0.7964,0.7427,0.0048,0.0026,0.5516,0.9981,478.342918,Siit_Pistachio,0
889,81407.0,1244.4410,497.2620,220.6327,0.8962,321.9482,0.9254,87965.0,0.5527,2.2538,0.6606,0.6474,0.0061,0.0027,0.4192,0.9447,460.166255,Kirmizi_Pistachio,1


## cross validation/tuning

In [17]:
import sklearn
from sklearn.model_selection import StratifiedKFold, cross_validate
from typing import Dict

def cross_validate_estimator(train_X: pd.DataFrame, train_Y: pd.DataFrame, clf: sklearn.base.BaseEstimator, metrics: Dict, n_folds: int=5, cv_seed:int=23, n_jobs: int=2):
    """for a given set of model parameters, use cross validation to evaluate model performance"""
    
    # generate cv_folds
    cv_folds = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=cv_seed)

    results = cross_validate(clf, train_X, train_Y, cv=cv_folds, scoring=metrics, n_jobs=n_jobs)

    return results
    


In [20]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, make_scorer

params = {
    'learning_rate': 0.1,
    'booster': 'gbtree',
    'n_jobs': 1,
    'gamma': 0.01,
    'min_child_weight': 0.05,
    'max_depth': 5,
    'subsample': 0.8,
    'colsample_bytree': 0.3,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1
}

cv_seed = 37
metrics = {
    "roc_auc": make_scorer(roc_auc_score),
    "precision": make_scorer(precision_score),
    "recall": make_scorer(recall_score),
    "f1_score": make_scorer(f1_score)
}

cols = train_proc.columns
features = [x for x in cols if x not in ['Class','Target']]
Xx = train_proc[features]
Yy = train_proc.Target.values

clf = XGBClassifier(objective='binary:logistic', eval_metric='auc', **params)

results = cross_validate_estimator(Xx, Yy, clf, metrics, cv_seed=cv_seed, n_jobs=2)
agged_results = {k:(np.mean(v), np.std(v)) for k,v in results.items()}

agged_results


{'fit_time': (0.12704544067382811, 0.0031564798201697618),
 'score_time': (0.00627593994140625, 0.00024546266215500455),
 'test_roc_auc': (0.8691124721914356, 0.010111355494033258),
 'test_precision': (0.8864885217567421, 0.011389116123450178),
 'test_recall': (0.8923857868020304, 0.031684236617923335),
 'test_f1_score': (0.8889775847285855, 0.013208642189853053)}

In [27]:
from bayes_opt import BayesianOptimization
from bayes_opt.util import Colours

fixed_parms = {
    "booster": "gbtree",
    "n_jobs": 1,
    'colsample_bytree': 0.3
}
pbounds = {
    'learning_rate': (0.01, 0.3),
    'gamma': (0.0, 0.3),
    'min_child_weight': (0.01, 0.07),
    'max_depth': (3, 5),
    'subsample': (0.7, 0.9),
    'reg_alpha': (0.01, 0.1),
    'reg_lambda': (0.01, 0.1)
}

def optimise_tune(train_x, train_y, pbounds, fixed_parms, metrics, cv_seed, n_folds: int=5):

    trials = []

    def run_trial(**probe_params):
        params = {**probe_params, **fixed_parms}
        for x in ['max_depth']:
            if x in params:
                params[x] = int(params[x])
        
        clf = XGBClassifier(objective='binary:logistic', eval_metric='auc', **params)

        results = cross_validate_estimator(train_x, train_y, clf, metrics, cv_seed=cv_seed, n_jobs=2)

        agged_results = {k:(np.mean(v), np.std(v)) for k,v in results.items()}

        final_score = agged_results['test_roc_auc'][0] - agged_results['test_roc_auc'][1]/np.sqrt(n_folds)

        

        trials.append( {"final_score": final_score, 'params': params, "results": agged_results})
        return final_score

    optimizer = BayesianOptimization(
        f = run_trial,
        random_state=43,
        pbounds=pbounds,
        verbose=2)

    optimizer.maximize(n_iter=10)

    print(f"best_result: {optimizer.max}")
    return optimizer.max, trials

optimise_tune(Xx, Yy, pbounds, fixed_parms, metrics, cv_seed, n_folds=5)

    
        

    



|   iter    |  target   |   gamma   | learni... | max_depth | min_ch... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------
| 1         | 0.8634    | 0.03452   | 0.1866    | 3.267     | 0.02444   | 0.03944   | 0.08732   | 0.8332    |
| 2         | 0.8629    | 0.1623    | 0.01841   | 4.467     | 0.0337    | 0.08218   | 0.0329    | 0.7114    |
| 3         | 0.8681    | 0.26      | 0.0741    | 3.81      | 0.02897   | 0.0169    | 0.08589   | 0.8698    |
| 4         | 0.868     | 0.2914    | 0.1218    | 4.909     | 0.03675   | 0.07028   | 0.01743   | 0.8794    |
| 5         | 0.8751    | 0.0894    | 0.08607   | 3.01      | 0.04259   | 0.0528    | 0.06727   | 0.8956    |
| 6         | 0.8742    | 0.2186    | 0.2439    | 3.573     | 0.04407   | 0.0684    | 0.09759   | 0.8835    |
| 7         | 0.8722    | 0.0875    | 0.1051    | 3.508     | 0.03248   | 0.05424   | 0.01801   | 0.8052    |
| 8       

({'target': 0.8751419547707581,
  'params': {'gamma': 0.08940105020662881,
   'learning_rate': 0.0860683985529506,
   'max_depth': 3.010259098128914,
   'min_child_weight': 0.04259215091682436,
   'reg_alpha': 0.05280367303065279,
   'reg_lambda': 0.06727363098143704,
   'subsample': 0.8956408268164012}},
 [{'final_score': 0.8634094710132025,
   'params': {'gamma': 0.03451636991693369,
    'learning_rate': 0.1866292963910496,
    'max_depth': 3,
    'min_child_weight': 0.024435377197920927,
    'reg_alpha': 0.03944251502300258,
    'reg_lambda': 0.0873223741853738,
    'subsample': 0.8332180426196052,
    'booster': 'gbtree',
    'n_jobs': 1,
    'colsample_bytree': 0.3},
   'results': {'fit_time': (0.08314971923828125, 0.0018545725583630284),
    'score_time': (0.005869054794311523, 4.912223565448017e-05),
    'test_roc_auc': (0.8698113818923021, 0.014315107911562121),
    'test_precision': (0.8871423614402811, 0.011757889032921265),
    'test_recall': (0.8923857868020304, 0.027505009

## Train a model

In [10]:
from xgboost import XGBClassifier

cols = train_proc.columns
features = [x for x in cols if x not in ['Class','Target']]
Xx = train_proc[features]
Yy = train_proc.Target.values

params = {
    'learning_rate': 0.1,
    'booster': 'gbtree',
    'n_jobs': 1,
    'gamma': 0.01,
    'min_child_weight': 0.05,
    'max_depth': 5,
    'subsample': 0.8,
    'colsample_bytree': 0.3,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1
}
    
clf = XGBClassifier(objective='binary:logistic', eval_metric='auc', **params)

    
    

model = clf.fit(Xx,Yy)


    


In [41]:
model.get_xgb_params()

{'objective': 'binary:logistic',
 'base_score': None,
 'booster': 'gbtree',
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': 0.3,
 'eval_metric': 'auc',
 'gamma': 0.01,
 'gpu_id': None,
 'grow_policy': None,
 'interaction_constraints': None,
 'learning_rate': 0.1,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': 5,
 'max_leaves': None,
 'min_child_weight': 0.05,
 'monotone_constraints': None,
 'n_jobs': 1,
 'num_parallel_tree': None,
 'predictor': None,
 'random_state': None,
 'reg_alpha': 0.1,
 'reg_lambda': 0.1,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': 0.8,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [11]:
train_result = model.predict_proba(Xx)
train_result[0:5]

array([[0.99292594, 0.00707408],
       [0.25227547, 0.74772453],
       [0.00805604, 0.99194396],
       [0.9967752 , 0.00322476],
       [0.01356578, 0.9864342 ]], dtype=float32)

In [ ]:
# for x in train.columns:
#     schema = schema.update_column(x, nullable=False)
sc2 = pa.DataFrameSchema({
    'col1': pa.Column(str, nullable=True),
    'col2': pa.Column(int, nullable=False, unique=True)})
print(sc2)
    
    

In [ ]:
print(schema)

In [ ]:
#print(schema.to_script())

In [48]:
# import pandera
# pandera.__version__
import xgboost
xgboost.__version__

'1.7.6'